In [1]:
import os
from pathlib import Path
import re
from datetime import datetime

def rename_ndvi_files(base_folders: list, dry_run: bool = False) -> None:
    """
    Rename NDVI files in multiple folders from 'img_infer_*_YYYY-MM-DD.tif' to 'ndvi8days_YYYY-MM-DD.tif'
    
    Args:
        base_folders: List of base folder paths
        dry_run: If True, only print planned actions without executing them
    """
    # Compile regex pattern to extract date
    date_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})\.tif$')
    
    for base_folder in base_folders:
        ndvi_folder = os.path.join(base_folder, 'ndvi')
        
        if not os.path.exists(ndvi_folder):
            print(f"\nSkipping: NDVI folder not found in {base_folder}")
            continue
            
        print(f"\nProcessing folder: {ndvi_folder}")
        
        # Get all tif files in the ndvi folder
        tif_files = list(Path(ndvi_folder).glob('img_infer_*.tif'))
        
        if not tif_files:
            print("No matching files found.")
            continue
            
        for file_path in tif_files:
            # Extract date from filename
            date_match = date_pattern.search(file_path.name)
            if date_match:
                date_str = date_match.group(1)
                new_name = f'ndvi8days_{date_str}.tif'
                new_path = file_path.parent / new_name
                
                print(f"Found: {file_path.name}")
                print(f"Will rename to: {new_name}")
                
                if not dry_run:
                    try:
                        file_path.rename(new_path)
                        print(f"Renamed successfully")
                    except Exception as e:
                        print(f"Error renaming file: {str(e)}")
                
            else:
                print(f"Skipping {file_path.name}: Could not extract date")

# Example usage
if __name__ == "__main__":
    # List of base folders to process
    folders = [
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/GiaiXuan-TanKy-NgheAn",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/HiepHoa-KinhMon-HaiDuong",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/ThanhBinh-TrangBom-DongNai",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/ThuyVan-ThaiThuy-ThaiBinh"
        # Add more folders as needed
    ]
    
    # First do a dry run to see what would happen
    print("Performing dry run...")
    rename_ndvi_files(folders, dry_run=True)
    
    # Ask for confirmation before proceeding
    response = input("\nDo you want to proceed with the actual renaming? (yes/no): ")
    if response.lower() == 'yes':
        print("\nRenaming files...")
        rename_ndvi_files(folders, dry_run=False)
    else:
        print("\nOperation cancelled.")

Performing dry run...

Processing folder: /mnt/storage/data/EOV_LST/Train_LST/Data_train_1/GiaiXuan-TanKy-NgheAn/ndvi
Found: img_infer_giaixuan_2022-10-01.tif
Will rename to: ndvi8days_2022-10-01.tif
Found: img_infer_giaixuan_2022-10-17.tif
Will rename to: ndvi8days_2022-10-17.tif
Found: img_infer_giaixuan_2022-11-02.tif
Will rename to: ndvi8days_2022-11-02.tif
Found: img_infer_giaixuan_2022-11-18.tif
Will rename to: ndvi8days_2022-11-18.tif
Found: img_infer_giaixuan_2022-12-04.tif
Will rename to: ndvi8days_2022-12-04.tif
Found: img_infer_giaixuan_2022-12-20.tif
Will rename to: ndvi8days_2022-12-20.tif
Found: img_infer_giaixuan_2023-01-05.tif
Will rename to: ndvi8days_2023-01-05.tif
Found: img_infer_giaixuan_2023-01-21.tif
Will rename to: ndvi8days_2023-01-21.tif
Found: img_infer_giaixuan_2023-02-06.tif
Will rename to: ndvi8days_2023-02-06.tif
Found: img_infer_giaixuan_2023-02-22.tif
Will rename to: ndvi8days_2023-02-22.tif
Found: img_infer_giaixuan_2023-03-10.tif
Will rename to: ndvi8